In [9]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str
    state:str

In [16]:
person=Person(name="vijay", age=35, city="Pune", state="MH",)
person


Person(name='vijay', age=35, city='Pune', state='MH')

In [17]:
### int can be coverted to str when we use @dataclass
### no hard typecast check
person1=Person(name="vijay", age=35, city="Pune", state=1)
person1

Person(name='vijay', age=35, city='Pune', state=1)

In [18]:
### int can be coverted to str when we use @dataclass
### no hard typecast check
person1=Person(name="vijay", age="35", city="Pune", state=1)
person1

Person(name='vijay', age='35', city='Pune', state=1)

#### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.



In [19]:
## Pydantic
from pydantic import BaseModel

In [20]:
class Person01(BaseModel):
    name:str
    age:int
    city:str
    state:str

In [21]:
## With Pydantic strict validation will be applied and state int can not converted to str
person01=Person01(name="vijay", age="35", city="Pune", state=1)
person01

ValidationError: 1 validation error for Person01
state
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [22]:
## we need pass state as str then it will work
person02=Person01(name="vijay", age="35", city="Pune", state="1")
person02

Person01(name='vijay', age=35, city='Pune', state='1')

#### 2. Model with Optional Fields
Add optional fields using Python's Optional type:



In [24]:
from typing import Optional

class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary: Optional[float]=None #Optional with default value
    is_active: Optional[bool]=True #optional field with default value

In [25]:
## default value will automatically assigned
emploeyee1=Employee(id=1,name="vijay",department="technology")
emploeyee1

Employee(id=1, name='vijay', department='technology', salary=None, is_active=True)

In [26]:
## given salary value will get assigned
emploeyee2=Employee(id=1,name="vijay",salary=10000,department="technology")
emploeyee2

Employee(id=1, name='vijay', department='technology', salary=10000.0, is_active=True)

### Lets play with List now

In [30]:
from typing import List

class Department(BaseModel):
    id:int
    employees:List[str]
    headCount:int

In [31]:
department1=Department(id=100,employees=("vijay","krish","prashant"),headCount=10)
department1

Department(id=100, employees=['vijay', 'krish', 'prashant'], headCount=10)

In [32]:
## as we have add int to List[str] , pydentic will throw error
department2=Department(id=100,employees=("vijay","krish",7),headCount=10)
department2

ValidationError: 1 validation error for Department
employees.2
  Input should be a valid string [type=string_type, input_value=7, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [33]:
## in production we dont want to throw this type error instend we will write in a readable format
## let use Exception handling here to achieve this
try:
    department3=Department(id=100,employees=("vijay","krish",7),headCount=10)
    department3
except Exception as e:
    print(e)

1 validation error for Department
employees.2
  Input should be a valid string [type=string_type, input_value=7, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### Model with Nested Models
Create complex structures with nested models:

In [35]:
from pydantic import BaseModel
from typing import List

class Student(BaseModel):
    rollno:int
    name:str
    gender:str

class Branch(BaseModel):
    barnch_name:str
    students:List[Student]

In [36]:
branch1=Branch(barnch_name="Mechanical",
               students=[
        {"rollno": 3, "name": "Charlie", "gender": "Male"},
        {"rollno": 4, "name": "Diana", "gender": "Female"}
    ]
    )
branch1

Branch(barnch_name='Mechanical', students=[Student(rollno=3, name='Charlie', gender='Male'), Student(rollno=4, name='Diana', gender='Female')])

#### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [37]:
from pydantic import BaseModel, Field

class Television(BaseModel):
    model:str=Field(min_length=3,max_length=10)
    price:float=Field(gt=5000,lt=100000)
    quatity:int=Field(gt=0)

In [38]:
## here we are applying condition base validation
try:
    tele1=Television(model="android",price=80000,quatity=0)
except Exception as e:
    print(e)

1 validation error for Television
quatity
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than


In [39]:
from pydantic import BaseModel, Field

class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=35,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")


user1 = User(username="vijay")
print(user1)

username='vijay' age=35 email='user@example.com'


In [40]:
user2 = User(username="vijay",age=36,email="vijay@gmail.com")
print(user2)

username='vijay' age=36 email='vijay@gmail.com'


### Lets create email validator now


In [41]:
## please intall pydantic[email] for EmailStr

from pydantic import BaseModel, Field, EmailStr

class User001(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=35,description="User age default to 18 ")
    email:EmailStr= Field(default_factory=lambda: "user@example.com",description="Default email address")


user01 = User001(username="vijay",age=36,email="vijay@gmail.com")
print(user01)

username='vijay' age=36 email='vijay@gmail.com'


In [42]:
## invalid email is throwing error
try:
    user02 = User001(username="vijay",age=36,email="vijay@.com")
    print(user02)
except Exception as e:
    print(e)

1 validation error for User001
email
  value is not a valid email address: An email address cannot have a period immediately after the @-sign. [type=value_error, input_value='vijay@.com', input_type=str]


In [43]:
## To print defination of your class
User001.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 35,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'format': 'email',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User001',
 'type': 'object'}